Some basic statistics about the data

In [9]:
!conda update -y dask

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::anaconda==5.2.0=py36_3
done

## Package Plan ##

  environment location: /home/ec2-user/anaconda3

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2019.03  |           py36_0           5 KB
    alabaster-0.7.12           |           py36_0          17 KB
    anaconda-custom            |           py36_1           3 KB
    anaconda-client-1.7.2      |           py36_0         141 KB
    anaconda-project-0.8.3     |             py_0         212 KB
    astroid-2.3.2              |           py36_0         283 KB
    astropy-3.2.2              |   py36h7b6447c_0         7.1 MB
    atomicwrites-1.3.0         |           py36_1          13 KB
    babel-2.7.0        

mccabe-0.6.1         | 14 KB     | ##################################### | 100% 
wurlitzer-1.0.3      | 14 KB     | ##################################### | 100% 
sphinxcontrib-qthelp | 26 KB     | ##################################### | 100% 
bkcharts-0.2         | 127 KB    | ##################################### | 100% 
cython-0.29.13       | 2.2 MB    | ##################################### | 100% 
sphinxcontrib-devhel | 23 KB     | ##################################### | 100% 
jupyterlab_server-1. | 26 KB     | ##################################### | 100% 
pycurl-7.43.0.2      | 60 KB     | ##################################### | 100% 
clyent-1.2.2         | 18 KB     | ##################################### | 100% 
_anaconda_depends-20 | 5 KB      | ##################################### | 100% 
jupyterlab-1.1.4     | 11.7 MB   | ##################################### | 100% 
numexpr-2.7.0        | 196 KB    | ##################################### | 100% 
mpc-1.1.0            | 94 KB

mpfr-4.0.1           | 575 KB    | ##################################### | 100% 
libcurl-7.61.0       | 494 KB    | ##################################### | 100% 
unicodecsv-0.14.1    | 25 KB     | ##################################### | 100% 
jeepney-0.4.1        | 21 KB     | ##################################### | 100% 
babel-2.7.0          | 5.8 MB    | ##################################### | 100% 
lazy-object-proxy-1. | 30 KB     | ##################################### | 100% 
cycler-0.10.0        | 13 KB     | ##################################### | 100% 
patchelf-0.9         | 71 KB     | ##################################### | 100% 
imagesize-1.1.0      | 9 KB      | ##################################### | 100% 
sphinxcontrib-1.0    | 3 KB      | ##################################### | 100% 
conda-verify-3.4.2   | 25 KB     | ##################################### | 100% 
numpydoc-0.9.1       | 31 KB     | ##################################### | 100% 
llvmlite-0.29.0      | 17.7 

In [15]:
!pip install dask[array] --upgrade

Requirement already up-to-date: dask[array] in /home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages (2.6.0)


In [ ]:
import pandas as pd
import psycopg2
from dask.distributed import Client
import dask.dataframe as dd

In [4]:
client = Client('Dask-Scheduler.local-dask:8786')

Solving environment: done

## Package Plan ##

  environment location: /home/ec2-user/anaconda3

  added / updated specs: 
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.7.12               |           py36_0         3.0 MB
    conda-package-handling-1.6.0|   py36h7b6447c_0         871 KB
    tqdm-4.36.1                |             py_0          50 KB
    ------------------------------------------------------------
                                           Total:         3.9 MB

The following NEW packages will be INSTALLED:

    conda-package-handling: 1.6.0-py36h7b6447c_0
    tqdm:                   4.36.1-py_0         

The following packages will be UPDATED:

    ca-certificates:        2019.5.15-0          --> 2019.10.16-0     
    certifi:                2019.6.16-py36_0     --> 2019.9.11-py36_0 
    conda:                  4.5.12-py36_0        --> 4.7.12-py3

pytz-2019.3          | 231 KB    | ##################################### | 100% 
click-7.0            | 118 KB    | ##################################### | 100% 
idna-2.7             | 132 KB    | ##################################### | 100% 
pyparsing-2.4.2      | 61 KB     | ##################################### | 100% 
packaging-19.2       | 30 KB     | ##################################### | 100% 
libvpx-1.7.0         | 2.4 MB    | ##################################### | 100% 
zict-1.0.0           | 12 KB     | ##################################### | 100% 
mkl_fft-1.0.14       | 173 KB    | ##################################### | 100% 
cytoolz-0.10.0       | 439 KB    | ##################################### | 100% 
jupyter_core-4.6.0   | 74 KB     | ##################################### | 100% 
freeglut-3.0.0       | 251 KB    | ##################################### | 100% 
attrs-19.3.0         | 39 KB     | ##################################### | 100% 
pip-19.3.1           | 1.9 M

In [7]:
client.get_versions(check=False)
client

Client Scheduler: tcp://Dask-Scheduler.local-dask:8786 Dashboard: http://Dask-Scheduler.local-dask:8787/status,Cluster Workers: 1 Cores: 2 Memory: 1.80 GB


In [15]:
db_host = '***REMOVED***'
db_port = 5432
db_name = "postgres"
db_user = "***REMOVED***"
db_pass = "***REMOVED***"
loiste_bbox = '25,62.7,31.4,66.4'
sssoy_bbox  = '24.5,60,30.6,63.5'

In [18]:
def get_polygons_over_network(start, end, bbox):
    """ Get polygons from db """

    conn = psycopg2.connect("dbname='%s' user='%s' host='%s' password='%s'" % (db_name, db_user, db_host, db_pass))
    
    sql = """
        SELECT weather_parameter,low_limit,high_limit,point_in_time 
        FROM sasse.stormcell 
        WHERE ST_MakeEnvelope({bbox}, 4326) && geom 
         AND point_in_time >= '{start}' AND point_in_time < '{end}'
         """.format(bbox=bbox,start=start,end=end)        

    cursor = conn.cursor()
    cursor.execute(sql)
    results = cursor.fetchall()

    print('Found {} records.'.format(len(results)))

    return results

In [20]:
polygons_loiste = get_polygons_over_network('2010-01-01', '2019-01-01',loiste_bbox)

Found 44857 records.
